In [ ]:
# Need to double check the appointment times
# Comb through daily openings a couple days out, and confirm whether there
# are overlapping shifts

In [2]:
# Loading libraries
import boto3
import pandas as pd
from io import StringIO
from math import floor
import matplotlib.pyplot as plt

In [3]:
# Defining paths
myBucket = 'tcmbooking'

In [4]:
# Function which returns lists of errors, openings and shifts in s3 bucket
def get_tcm_files(bucket):
    """Get a list of keys in an S3 bucket"""
    errors = []
    openings = []
    shifts = []
    
    resp = s3.list_objects_v2(Bucket=bucket)
    for x in resp['Contents']:
        if "error" in x['Key']:
            errors.append(x['Key'])
        elif "openings" in x['Key']:
            openings.append(x['Key'])
        elif "shifts" in x['Key']:
            shifts.append(x['Key'])
        
    return errors, openings, shifts

In [5]:
# Getting list of files in s3 bucket
s3 = boto3.client('s3')
errors, openings, shifts = get_tcm_files(myBucket)

In [142]:
# Creating data frame of the openings
dailyOpenings = pd.DataFrame()

for x in openings:
    # Reading in data from s3
    csv_obj = s3.get_object(Bucket=myBucket, Key=x)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string))
    
    # Selecting only same day data
    df = df.loc[df['DaysAhead']==0]
    dailyOpenings = dailyOpenings.append(df, ignore_index=True)
    
dailyOpenings.drop_duplicates(inplace=True)

In [143]:
# Creating data frame of the shifts
dailyShifts = pd.DataFrame()

for x in shifts:
    # Reading in data from s3
    csv_obj = s3.get_object(Bucket=myBucket, Key=x)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string))
    
    # Selecting only same day data
    df = df.loc[df['DaysAhead']==0]
    dailyShifts = dailyShifts.append(df, ignore_index=True)
    
dailyShifts.drop_duplicates(inplace=True)

In [144]:
# Getting the opening time
dailyShifts['StartH'] = dailyShifts['Start'].str[:2]
dailyShifts['StartM'] = dailyShifts['Start'].str[3:]
dailyShifts['StartH'] = dailyShifts.StartH.astype(float)
dailyShifts['StartM'] = dailyShifts.StartM.astype(float)
dailyShifts['StartM'] = dailyShifts['StartM'] / 60
dailyShifts['Open']  = dailyShifts['StartH'] + dailyShifts['StartM']
dailyShifts = dailyShifts.drop(['StartH', 'StartM'], axis = 1)

In [145]:
# Getting the closing time
dailyShifts['CloseH'] = dailyShifts['End'].str[:2]
dailyShifts['CloseM'] = dailyShifts['End'].str[3:]
dailyShifts['CloseH'] = dailyShifts.CloseH.astype(float)
dailyShifts['CloseM'] = dailyShifts.CloseM.astype(float)
dailyShifts['CloseM'] = dailyShifts['CloseM'] / 60
dailyShifts['Close']  = dailyShifts['CloseH'] + dailyShifts['CloseM']
dailyShifts = dailyShifts.drop(['CloseH', 'CloseM'], axis = 1)

In [146]:
# Getting the open hours per day
openHours = dailyShifts.groupby(['URL', 'Date'], as_index=False).agg({"Open":"min", "Close":"max"})
openHours['maxHours'] = (openHours['Close'] - openHours['Open'])*60
openHours = openHours.drop(['Open', 'Close'], axis = 1)

In [147]:
# Joining max hours to the daily shifts data frame
print('Number of rows pre-merge: ' + str(dailyShifts.shape[0]))
dailyShifts = dailyShifts.merge(openHours, on = ['URL', 'Date'], how='left', right_index=False)
print('Number of rows pre-merge: ' + str(dailyShifts.shape[0]))

Number of rows pre-merge: 658
Number of rows pre-merge: 658


In [148]:
# Function to choose the shift duration to be used in future calculations
def getHours(row):
    if row['Duration'] > row['maxHours']:
        val = row['maxHours']
    else:
        val = row['Duration']
        
    return val

In [149]:
dailyShifts['WorkTime'] = dailyShifts.apply(getHours, axis = 1)

In [150]:
# Aggregating shifts and openings
shifts_agg = dailyShifts.groupby(['Clinician', 'Date'], as_index=False).agg({"URL": "first", "WorkTime": "sum"})
openings_agg = dailyOpenings.groupby(['Clinician', 'Date'], as_index=False).agg({"URL": "first", "Duration": "sum", "DaysAhead": "count"})

# Renaming columns to eliminate duplicates
shifts_agg.columns = ['Clinician', 'Date', 'URL', 'ShiftTime']
openings_agg.columns = ['Clinician', 'Date', 'URL', 'OpenTime', 'OpenSlots']

In [151]:
# Merging shifts and openings data frames
print('Number of rows pre-merge: ' + str(shifts_agg.shape[0]))
daily = shifts_agg.merge(openings_agg, on = ['Clinician', 'URL', 'Date'], how='left', right_index=False)
daily['OpenTime'] = daily['OpenTime'].fillna(0)
daily['OpenSlots'] = daily['OpenSlots'].fillna(0)
print('Number of rows post-merge: ' + str(daily.shape[0]))

Number of rows pre-merge: 421
Number of rows post-merge: 421


In [152]:
# Getting price data
prices = pd.read_csv('/home/ubuntu/getTCM/jane_url.csv')
prices.columns = ['Clinic', 'Clinician', 'URL', 'Location', 'Cost', 'ApTime', 'Area']
prices['ApTime'] = prices['ApTime'].fillna(60)
prices = prices.drop(['Clinician'], axis=1)

In [153]:
# Joining data frames
daily = daily.merge(prices, on = 'URL', how='left', right_index=False)
daily.drop_duplicates(inplace=True)
print('Number of rows post-merge: ', str(daily.shape[0]))

Number of rows post-merge:  421


In [154]:
# Joining data frames
daily = daily.merge(openHours, on = ['URL', 'Date'], how='left', right_index=False)
daily.drop_duplicates(inplace=True)
print('Number of rows post-merge: ', str(daily.shape[0]))

Number of rows post-merge:  421


In [155]:
daily['Duration'] = daily['ShiftTime']
daily['ShiftTime'] = daily.apply(getHours, axis = 1)
daily = daily.drop(['Duration', 'maxHours'], axis=1)

In [156]:
# Estimating revenue
daily['AllSlots'] = daily['ShiftTime'] / daily['ApTime']
daily = daily.round({'Bookings': 0})

daily['AllSlots'] = daily['ShiftTime'] / daily['ApTime']
daily['AllSlots'] = daily.AllSlots.apply(floor)
daily['BookedSlots'] = daily['AllSlots'] - daily['OpenSlots']
daily['DailyRevenue'] = daily['BookedSlots'] * daily['Cost']

In [157]:
# Calculating week number
daily['Week'] = pd.to_datetime(daily['Date'])
daily['Week'] = daily['Week'].dt.week

In [893]:
clinicians = dailyOpenings.Clinician.unique()
print('There are ' + str(len(clinicians)) + ' clinicians')

There are 61 clinicians


In [1037]:
x = 60
y = 0
dailyX = daily[daily['Clinician']==clinicians[x]]
days = dailyX.Date.unique()
dailyX = dailyX[dailyX['Date']==days[y]]
dailyOpenX = dailyOpenings[(dailyOpenings['Clinician']==clinicians[x]) &  (dailyOpenings['Date']==days[y])]
dailyShiftX = dailyShifts[(dailyShifts['Clinician']==clinicians[x]) &  (dailyShifts['Date']==days[y])]
print('Showing ' + str(x) + ' of ' + str(len(clinicians)) + ' clinicians')
print('Showing ' + str(y) + ' of ' + str(len(dailyX)) + ' days')
dailyX

Showing 60 of 61 clinicians
Showing 0 of 1 days


,Clinician,Date,URL,ShiftTime,OpenTime,OpenSlots,Clinic,Location,Cost,ApTime,Area,AllSlots,BookedSlots,DailyRevenue,Week
7,Alyssa Huang,2019-03-08,https://elementswellness.janeapp.com/#/staff_member/4/treatment/5,570.0,300.0,5.0,Elements Wellness,1,95,60,Midtown,9,4.0,380.0,10


In [1038]:
dailyShiftX

,Clinician,URL,Today,DayOfWeek,Date,Start,End,Duration,DaysAhead,Open,Close,maxHours,WorkTime
307,Alyssa Huang,https://elementswellness.janeapp.com/#/staff_member/4/treatment/5,2019-03-08,Friday,2019-03-08,10:00,19:30,570,0,10.0,19.5,570.0,570


In [1039]:
dailyOpenX

,Clinician,URL,Today,DayOfWeek,Date,Start,End,Duration,DaysAhead
642,Alyssa Huang,https://elementswellness.janeapp.com/#/staff_member/4/treatment/5,2019-03-08,Friday,2019-03-08,10:00,11:00,60,0
643,Alyssa Huang,https://elementswellness.janeapp.com/#/staff_member/4/treatment/5,2019-03-08,Friday,2019-03-08,12:00,13:00,60,0
644,Alyssa Huang,https://elementswellness.janeapp.com/#/staff_member/4/treatment/5,2019-03-08,Friday,2019-03-08,13:00,14:00,60,0
645,Alyssa Huang,https://elementswellness.janeapp.com/#/staff_member/4/treatment/5,2019-03-08,Friday,2019-03-08,15:00,16:00,60,0
646,Alyssa Huang,https://elementswellness.janeapp.com/#/staff_member/4/treatment/5,2019-03-08,Friday,2019-03-08,16:00,17:00,60,0


In [1027]:
dailyOpenings[dailyOpenings['Clinician']==clinicians[56]]

,Clinician,URL,Today,DayOfWeek,Date,Start,End,Duration,DaysAhead
370,Chris Jeffrey,https://chris-jeffrey-acupuncture.janeapp.com/#/staff_member/1/treatment/3,2019-03-06,Wednesday,2019-03-06,09:00,10:15,75,0
371,Chris Jeffrey,https://chris-jeffrey-acupuncture.janeapp.com/#/staff_member/1/treatment/3,2019-03-06,Wednesday,2019-03-06,10:15,11:30,75,0
372,Chris Jeffrey,https://chris-jeffrey-acupuncture.janeapp.com/#/staff_member/1/treatment/3,2019-03-06,Wednesday,2019-03-06,16:00,17:15,75,0
373,Chris Jeffrey,https://chris-jeffrey-acupuncture.janeapp.com/#/staff_member/1/treatment/3,2019-03-06,Wednesday,2019-03-06,19:00,20:15,75,0
529,Chris Jeffrey,https://chris-jeffrey-acupuncture.janeapp.com/#/staff_member/1/treatment/3,2019-03-07,Thursday,2019-03-07,12:30,13:45,75,0
530,Chris Jeffrey,https://chris-jeffrey-acupuncture.janeapp.com/#/staff_member/1/treatment/3,2019-03-07,Thursday,2019-03-07,13:45,15:00,75,0
531,Chris Jeffrey,https://chris-jeffrey-acupuncture.janeapp.com/#/staff_member/1/treatment/3,2019-03-07,Thursday,2019-03-07,15:00,16:15,75,0
532,Chris Jeffrey,https://chris-jeffrey-acupuncture.janeapp.com/#/staff_member/1/treatment/3,2019-03-07,Thursday,2019-03-07,16:15,17:30,75,0
533,Chris Jeffrey,https://chris-jeffrey-acupuncture.janeapp.com/#/staff_member/1/treatment/3,2019-03-07,Thursday,2019-03-07,18:45,20:00,75,0
534,Chris Jeffrey,https://chris-jeffrey-acupuncture.janeapp.com/#/staff_member/1/treatment/3,2019-03-07,Thursday,2019-03-07,20:00,21:15,75,0
